In [1]:
from pyspark import SparkContext, SparkConf 
import numpy as np 
import operator

Creating a Spark Context with Application name and master core

In [2]:
conf=SparkConf().setAppName("Lab4-rdd").setMaster("local[*]")
sc = SparkContext(conf=conf)

Importing gzip, opening the cite75_99.txt gzip file and reading the gzip file and writing it to RDD.  

In [3]:
import gzip
with gzip.open('cite75_99.txt.gz', 'r') as f:
    rddCitations = sc.parallelize( f.readlines()[-800000:] )

Importing gzip, opening the patent gzip file and reading the gzip file and writing it to RDD.  

In [4]:
with gzip.open('apat63_99.txt.gz','r') as f:
    rddPatents = sc.parallelize( f.readlines()[-8000000:] ) 

Converting the rddCitation file in byte format to string format 

In [5]:
x = rddCitations.map(lambda x: x.decode('utf-8'))

Converting the rddPatents file in byte format to string format

In [6]:
y = rddPatents.map(lambda x: x.decode('utf-8'))

Splitting the string citation file by ',' and stripping of the leading and trailing white spaces

In [7]:
split_cit = x.map(lambda y: y.strip().split(','))

Splitting the string patent file by ',' and stripping of the leading and trailing white spaces, filtering out the null spaces present in the state column and considering only 1st and 5th column

In [8]:
split_pat = y.map(lambda x: x.strip().split(','))\
            .filter(lambda y: y[5] != '""')\
            .filter(lambda y : y[0] != '"PATENT"')\
            .map(lambda y: (y[0], y[5]))


Splitting the string patent file by ',' and stripping of the leading and trailing white spaces to be taken into consideration at the last

In [9]:
split_pat1 = y.map(lambda x: x.strip().split(','))\
              .filter(lambda y : y[0] != '"PATENT"')\

Testing the display of the variable assigned above

In [10]:
split_pat.take(4)

[('3070802', '"TX"'),
 ('3070803', '"IL"'),
 ('3070804', '"OH"'),
 ('3070805', '"CA"')]

In [11]:
split_pat1.take(4)

[['3070801',
  '1963',
  '1096',
  '',
  '"BE"',
  '""',
  '',
  '1',
  '',
  '269',
  '6',
  '69',
  '',
  '1',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['3070802',
  '1963',
  '1096',
  '',
  '"US"',
  '"TX"',
  '',
  '1',
  '',
  '2',
  '6',
  '63',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['3070803',
  '1963',
  '1096',
  '',
  '"US"',
  '"IL"',
  '',
  '1',
  '',
  '2',
  '6',
  '63',
  '',
  '9',
  '',
  '0.3704',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['3070804',
  '1963',
  '1096',
  '',
  '"US"',
  '"OH"',
  '',
  '1',
  '',
  '2',
  '6',
  '63',
  '',
  '3',
  '',
  '0.6667',
  '',
  '',
  '',
  '',
  '',
  '',
  '']]

In [12]:
split_cit.take(1)

[['5935430', '5453769']]

Switching the citation and cited by using the lambda fucntion 

In [13]:
split_cited = split_cit.map(lambda y : (y[1],y[0]))

Joining split_cited database and split_pat database and storing it in variable cited_pat in RDD form.

In [14]:
cited_pat = split_cited.join(split_pat)

In [ ]:
cited_pat.take(3)

Sorting cited_pat by the second element and storing it in variable cited_pat_sort

In [ ]:
cited_pat_sort = cited_pat.sortBy(lambda x : x[1])

Joining the cited_pat_sort with split_pat dataset as the data sets dervied above, sorting it by the Key and storing it in intermediate table

In [ ]:
inter = cited_pat_sort.join(split_pat).sortBy(lambda x: x[0])

Taking the first values of intermediate table and displaying it

In [ ]:
inter.take(5)

Filtering out the same state citation using the lambda function and keeping a count by the key.

In [ ]:
final_count = inter.filter(lambda x : x[1][0][1] == x[1][1]).countByKey()

sorting the final by the count in descending order and storing it in variable count_final

In [ ]:
count_final = sorted(final_count.items(), key=lambda x: x[1], reverse=True)

Converting the list to RDD datatset 

In [ ]:
rddcount = sc.parallelize(count_final)

Displaying the top 10 counts in descending order for patents having same state citations

In [ ]:
rddcount.take(10)

sorting the final by the count in ascending order and storing it in variable count_final1

In [ ]:
count_final1 = sorted(final_count.items(), key=lambda x: x[1], reverse=False)

Converting the list to RDD datatset 

In [ ]:
rddcount1 = sc.parallelize(count_final1)

Add the count to the patent database as mentioned in the sample output

In [ ]:
split_pat1.join(rddcount1).cache()